<h1>Crawlers and Scrapers</h1>

The goal of this session is to build and run our own Amazon.com scraper using the **scrapy** python library. 

Our scraper will crawl through a specific product's customer review pages and get all of the available ratings and reviews. This will allow us to get complete review details that we can’t get with the Amazon Product Advertising API.

First we will install scrapy using pip command in terminal/cmd:

In [ ]:
!pip install scrapy

Next, we'll use scrapy to automatically generate a skeleton of the code needed for our scraper. (On terminal/cmd type the following command without the exclamation mark):

In [ ]:
!scrapy genspider amazon_scraper amazon.com

A new python-script, amazon_scraper.py file will be created. The final content of our scraper will be as follows:

In [ ]:
# -*- coding: utf-8 -*-
import scrapy

class AmazonScraperSpider(scrapy.Spider):
    name = 'amazon_scraper'
    allowed_domains = ['amazon.com']
    # assing a product-review-page url below
    start_urls = ['https://www.amazon.com/Apple-iPhone-Verizon-Unlocked-Renewed/product-reviews/B07HYDFX8G/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=helpful&pageNumber=1']
    
    def parse(self, response):
        review_texts = response.css('.a-size-base.review-text')
        for i in range(len(review_texts)):
            review_texts[i] = "".join(review_texts[i].css('::text').extract()).strip()

        review_ratings = response.css('[data-hook="review-star-rating"] > span::text').extract()

        for i in range(len(review_texts)):
            review = {
                'text' : review_texts[i],
                'rating': review_ratings[i]
            }
            yield review

        next_page_url = response.css('.a-last > a::attr(href)').extract_first()
        yield response.follow(next_page_url, self.parse)


We can call the script from terminal/cmd as follows:


In [ ]:
!scrapy runspider amazon_scraper.py -o out.json